In [100]:
'''
0.写一个爬虫，爬取豆瓣的电影(热门电影)信息
    -- 爬取的内容包括( 电影名称，导演姓名，演员姓名，电影类型，制片国家，日期，豆瓣评分)
    -- 并保存为“豆瓣电影数据.xlsx”输出
    提示: 
        * 前期调研时最好是能找到一个获取单个影片连接的入口(可以是通过翻页去拼接获取；也可以直接调取接口数据；)
            -- 这里我用的是后一种方式
        * 爬取时候由于访问量过大的原因，豆瓣蜀黍 会封掉，小伙伴的IP这里提供两种思路
            -- 爬的时候设置一个等待时间可以利用time.sleep()方法
            -- 代理，代理 requests模块已经为你们准好了，直接用吧
        * 在数据存储的时候发现数据只有270条
            -- 不要捉鸡，热门电影 只有270部 所以正常啦！
            -- 开始ND以为是豆瓣蜀黍的反爬机制(心中暗自叫好：“高手！~”)其实还好 ！哈哈哈哈
    
1.读取数据，以 豆瓣评分 为标准，看看电影评分分布 以及烂片情况(要求)：
    -- 读取数据文件 豆瓣电影数据.xlsx
    -- 查看 豆瓣评分 数据分布，绘制直方图 和 箱型图
    -- 判断 豆瓣评分 数据是否符合 正太分布
    -- 如果符合正太分布，这里以上四分位(该样本中所有数值由小到大排列后第 25% 的数字) 评分为 烂片标准
    -- 筛选出 烂片数据 并做排名，找到TOP20
    提示：
        * 读取完数据之后，去除缺失值
        * 这里可以用 ks 检验来判断数据是否符合正太分布
        
 2.什么题材的电影烂片最多(要求)？
    -- 按照 电影类型 字段分类，筛选出不同电影属于什么题材
    -- 整理数据 按照 题材 汇总，查看不同题材的烂片比例 并选取TOP10
    -- 将得到的题材烂片比例的TOP10制作散点图(横坐标为 题材 类型，纵坐标为 烂片比例，点大小为样本数量)
        * 用bokeh制图；按照烂片比例做降序排列；
    提示：
        * 删除 电影类型 字段空值的数据
        * 由于电影类型 会有多个，这里需要将一个电影每个类型都识别出来，在统计每个题材时都需要计算 例如：
            -- 如果一个电影类型为：“喜剧/爱情”，则在计算 喜剧、爱情题材的烂片比例时，都需要将该电影算上
        * 注意类型字段中，要删除空格字符
        * bokeh图设置点大小，这里通过 开放减小数据差距 size = count**0.5*系数   
        
3.和什么国家合拍更可能产生烂片(要求)？
    -- 按照制片国家/地区 字段分类，筛选不同电影的制片地
    -- 整理数据 按照题材汇总，查看不同题材的烂片比例
    提示：
        * 删除 制片国家/地区 字段空值的数据
        * 删除 制片国家/地区 中不包括 中国大陆的数据
        * 制片地删除 中国大陆 中国 台湾 香港 等噪音数据
        * 筛选合作电影 大于等于3部以上的国家
        
4.卡司数量是否和烂片有关(要求)？
    -- 计算每部电影的主演人数
    -- 按照主演人数分类，统计烂片率
        * 分类：1-2人，3-4人，5-6人，7-9人，10人以上
    -- 查看烂片比例最高的演员 TOP20
    提示：
        * 通过 主演 字段内做分列 来计算主演人数
        * 需要分别统计不同主演人数的电影数量以及烂片数量，再计算烂片比例
        * 这里可以按照明星再查看一下 他们的烂片率，比如 吴亦凡、范冰冰、杨幂、黄晓明....
        
5.不同导演每年电影产量情况是如何的(要求)？
    -- 通过 上映日期 筛选出每个电影的上映年份
    -- 查看不同导演的烂片比例、这里去除拍过 10次电影以下的导演
    -- 查看不同导演每年的电影产量制作散点图 (横坐标为年份，纵坐标为每年电影平均分，点大小该年电影数量)
        * 用bokeh制图；横坐标限定为2007-2017年；绘制散点图只需要产出烂片的导演数据；
    提示：
        * 注意要删除 上映日期 中的空格字符
        * 绘制图表时，分开建立数据绘制
'''



'\n0.写一个爬虫，爬取豆瓣的电影信息\n    -- 爬取的内容包括( 电影名称，导演姓名，演员姓名，电影类型，制片国家，日期，豆瓣评分)\n    -- 并保存为输出\n    \n1.读取数据，以 豆瓣评分 为标准，看看电影评分分布 以及烂片情况(要求)：\n    -- 读取数据文件 moviedata.xlsx\n    -- 查看 豆瓣评分 数据分布，绘制直方图 和 箱型图\n    -- 判断 豆瓣评分 数据是否符合 正太分布\n    -- 如果符合正太分布，这里以上四分位(该样本中所有数值由小到大排列后第 25% 的数字) 评分为 烂片标准\n    -- 筛选出 烂片数据 并做排名，找到TOP20\n    提示：\n        * 读取完数据之后，去除缺失值\n        * 这里可以用 ks 检验来判断数据是否符合正太分布\n        \n2.什么题材的电影烂片最多(要求)？\n    -- 按照 类型 字段分类，筛选出不同电影属于什么题材\n    -- 整理数据 按照 题材 汇总，查看不同题材的烂片比例 并选取TOP20\n    -- 将得到的题材烂片比例的TOP20制作散点图(横坐标为 题材 类型，纵坐标为 烂片比例，点大小为样本数量)\n        * 用bokeh制图；按照烂片比例做降序排列；\n    提示：\n        * 删除 类型 字段空值的数据\n        * 由于电影类型 会有多个，这里需要将一个电影每个类型都识别出来，在统计每个题材时都需要计算 例如：\n            -- 如果一个电影类型为：“喜剧/爱情”，则在计算 喜剧、爱情题材的烂片比例时，都需要将该电影算上\n        * 注意类型字段中，要删除空格字符\n        * bokeh图设置点大小，这里通过 开放减小数据差距 size = count**0.5*系数\n        \n3.和什么国家合拍更可能产生烂片(要求)？\n    -- 按照制片国家/地区 字段分类，筛选不同电影的制片地\n    -- 整理数据 按照题材汇总，查看不同题材的烂片比例\n    提示：\n        * 删除 制片国家/地区 字段空值的数据\n        * 删除 制片国家/地区 中不包括 中国大陆的数据\n

In [149]:
'''
0.写一个爬虫，爬取豆瓣的电影信息
    -- 爬取的内容包括( 电影名称，导演姓名，演员姓名，电影类型，制片国家，日期，豆瓣评分)
    -- 并保存为“豆瓣电影数据.xlsx”输出
    提示:
'''
import requests
import random
import json
from lxml import etree
import pandas as pd
import os
import time

os.chdir(r'F:\data1')


def capture_mv_info(url):
    mov_info = {'name': '',
                'dirname': '',
                'actorname': '',
                'movietype': '',
                'country': '',
                'date': '',
                'grade': ''}
    middle_list = {}
    html_1 = requests.get(url,proxies={'https:':random.choice(proxies_list)},headers=header)
    obj_html = etree.HTML(html_1.text)

    # 定义一个清洗国家的函数
    def country_clear(path):
        country = obj_html.xpath(path)[7:10]
        ct = []
        for item in country:
            if item.strip() != '':
                ct.append(item)
        return ct

    # 定义一个循环提取信息的函数
    def data_tq(path):
        data = []
        info = obj_html.xpath(path)
        for actor in info:
            data.append(actor.text)
        return data

    dirname = data_tq(r'//div[@id="info"]//a[@rel="v:directedBy"]')
    actorname = data_tq(r'//div[@id="info"]//span[@class="actor"]//a')
    movietype = data_tq(r'//div[@id="info"]//span[@property="v:genre"]')
    country = country_clear(r'//div[@id="info"]/text()')
    date = data_tq(r'//div[@id="info"]//span[@property="v:initialReleaseDate"]')
    name = data_tq('//div[@id="content"]//span[@property="v:itemreviewed"]')
    grade = data_tq('//div[@id="interest_sectl"]//strong[@class="ll rating_num"]')

    mov_info['name'] = name
    mov_info['dirname'] = dirname
    mov_info['actorname'] = actorname
    mov_info['movietype'] = movietype
    mov_info['country'] = country
    mov_info['date'] = date
    mov_info['grade'] = grade

    return mov_info

# 设置代理
global proxies_list
global header

proxies_list = ['101.37.79.125:3128','120.92.74.189:3128','120.92.74.237:3128','119.31.210.170:7777','125.62.26.197:3128']
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36'
}

url = r'https://movie.douban.com/j/search_subjects?type=movie&tag=热门&sort=recommend&page_limit=2000&page_start=1'
html_response = requests.get(url,proxies={'https:':random.choice(proxies_list)},headers=header)
html_text = html_response.text
print(len(html_text))
html_json = json.loads(html_text)

movie_url = []
for item in html_json['subjects']:
    movie_url.append(item['url'])


# 自定义一个函数 遍历url 获取电影信息
def movie_infos(url_list):
    mov_info_list = []
    num = 0
    for i in url_list:
        time.sleep(2)
        mov_info = capture_mv_info(i)
        mov_info_list.append(mov_info)
        num += 1
        print('第--%s--部--OK---' %num)
    return mov_info_list


mv = movie_infos(movie_url)

'''
 存储数据
'''
data = pd.DataFrame(mv)

writer = pd.ExcelWriter('豆瓣电影数据.xlsx')
data.to_excel(writer, 'sheet1', index=False)
writer.save()

print('finish!')

AttributeError: 'str' object has no attribute 'get'

In [51]:
'''
1.读取数据，以 豆瓣评分 为标准，看看电影评分分布 以及烂片情况(要求)：
    -- 读取数据文件 豆瓣电影数据.xlsx
    -- 查看 豆瓣评分 数据分布，绘制直方图 和 箱型图
    -- 判断 豆瓣评分 数据是否符合 正太分布
    -- 如果符合正太分布，这里以上四分位(该样本中所有数值由小到大排列后第 25% 的数字) 评分为 烂片标准
    -- 筛选出 烂片数据 并做排名，找到TOP20
    提示：
        * 读取完数据之后，去除缺失值
        * 这里可以用 ks 检验来判断数据是否符合正太分布
'''
import pandas as pd
import os


os.chdir(r'F:\data1')
df = pd.read_excel('豆瓣电影数据.xlsx',sheet_name=0)

# 清洗数据
def clear_data(data,args):
    for arg in args:
        df[arg] = df[arg].str.replace('[','').str.replace(']','').str.replace("'","")
data = clear_data(df,list(df.columns))

# 特殊字段需要需要单独清洗
df['grade'] = pd.to_numeric(df['grade'])
df['date'] = df['date'].str[:4]

#整理列名 并做进一步的清洗
df.columns = ['演员姓名','制片国家','上映日期','导演姓名','豆瓣评分','电影类型','电影名称']
df = df[df['豆瓣评分']>0]

# 计算分位差(划分整个样本数据中 烂片线)
fwc = df['豆瓣评分'].describe()
pass_line = fwc.loc['25%']
print('-烂片分割线',pass_line)

# 进行ks 正太检验(计算出来的值如果大于0.05 则为正太分布数据)
from scipy import stats
u = df['豆瓣评分'].mean()
std = df['豆瓣评分'].std()
print(stats.kstest(df['豆瓣评分'],'norm',(u,std)))

# 选出TOP20的烂片
data_lp = df[df['豆瓣评分']< pass_line].reset_index()
lp_Top20 = data_lp[['演员姓名','上映日期','导演姓名','豆瓣评分','电影名称']].sort_values(by='豆瓣评分').iloc[:20]

lp_Top20

-烂片分割线 5.9
KstestResult(statistic=0.07944859344772026, pvalue=0.06263659288764023)


,演员姓名,上映日期,导演姓名,豆瓣评分,电影名称
39,"玉木宏, 新木优子, 吉泽亮, 中村达也, 柄本明, 光石研",2018,中村哲平,3.9,恶与假面的规则 悪と仮面のルール
10,"韩庚, 凤小岳, 李媛, 山下智久, 廖启智, 迈克尔·帕帕约翰",2018,李海龙,4.0,解码游戏
56,"周渝民, 薛凯琪, 郑中基, 恬妞, 高捷, 盛朗熙",2017,谷德昭,4.1,天生不对
53,"张若昀, 马思纯, 李现, 刘敏涛, 姜超, 李霄云, 陈芋米, 安笑歌, 吴宥萱",2018,"李欣, 李洋",4.2,奇葩朵朵
30,"尚格·云顿, 杜夫·龙格尔, 杰思敏·沃兹, 帕特里克·基尔帕特里克, 阿尔·萨皮恩扎, 克...",2018,帕夏·帕特里基,4.3,深海越狱 Black Water
37,"韩庚, 杰瑞米·埃尔卡伊姆, 菅纫姿, 阿丽斯·德·朗克桑, 海伦·文森特, 雷诺·科恩, ...",2018,王超,4.3,寻找罗麦
49,"郭富城, 赵丽颖, 张翰, 任达华, 张蓝心, 朱一龙, 于晓光, 刘玥心, 奚美娟, 寇振...",2017,钟少雄,4.3,密战
50,"弗兰克·格里罗, 布鲁斯·威利斯, 乔纳森·斯卡奇, 奥莉维亚·库尔普, 泰勒·乔恩·奥尔森...",2018,布莱恩·A·米勒,4.3,报复 Reprisal
18,"吴君如, 沈腾, 岳云鹏, 张译, Papi酱, 吴镇宇, 方中信, 焦俊艳, 潘斌龙, 阿...",2017,吴君如,4.3,妖铃铃
23,"岳云鹏, 吴京, 吴秀波, 井柏然, 林志玲, 王宝强, 大鹏, 吴君如, 李晨, 杜淳, ...",2018,郭德纲,4.3,祖宗十九代


In [13]:
'''
 2.什么题材的电影烂片最多(要求)？
    -- 按照 电影类型 字段分类，筛选出不同电影属于什么题材
    -- 整理数据 按照 题材 汇总，查看不同题材的烂片比例 并选取TOP10
    -- 将得到的题材烂片比例的TOP10制作散点图(横坐标为 题材 类型，纵坐标为 烂片比例，点大小为样本数量)
        * 用bokeh制图；按照烂片比例做降序排列；
    提示：
        * 删除 电影类型 字段空值的数据
        * 由于电影类型 会有多个，这里需要将一个电影每个类型都识别出来，在统计每个题材时都需要计算 例如：
            -- 如果一个电影类型为：“喜剧/爱情”，则在计算 喜剧、爱情题材的烂片比例时，都需要将该电影算上
        * 注意类型字段中，要删除空格字符
        * bokeh图设置点大小，这里通过 开放减小数据差距 size = count**0.5*系数   
'''

import matplotlib.pyplot as plt
from bokeh.plotting import figure,show,output_file
from bokeh.models import ColumnDataSource,HoverTool
import os
#不发出警告
import warnings
warnings.filterwarnings('ignore') 
# 解决 jupyter notebook 中不能显示中文 
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False


# 第一步 筛选出所有题材的电影类型，查看不同的烂片比例
typelist = []
# 按照分隔符"," 进行类型分割(剔除为空的数据 和 数据前面的字符)
for i in df[df['电影类型'].notnull()]['电影类型'].str.replace(' ','').str.split(','):
    typelist.extend(i)
    
# 由于重复的内容较多 需要去重处理
typelist = list(set(typelist))

# 定义一个数据源
df_type = df[df['电影类型'].notnull()]

# 自定义一个函数 统计烂片信息(烂片名、烂片个数、烂片占比)
# 注：烂片占比 是指 这种类型的影片(豆瓣评分小于5.9) 占 该类型总量的百分比
# 这样就可以 推论出 那种类型的电影更容易出烂片
def lp_info(data,typei):
    dic_type_lp = {}
    datai = data[data['电影类型'].str.contains(typei)] # 筛选初属于该题材的类型数据
    lp_pre_i = len(datai[datai['豆瓣评分']< pass_line ]) / len(datai)
    # 总结输出烂片信息
    dic_type_lp['typename'] = typei
    dic_type_lp['typecount'] = len(datai)
    dic_type_lp['type_lp_pre'] = lp_pre_i
    
    return dic_type_lp

# 产出烂片信息
list_type_lp = []
for i in typelist:
    dici = lp_info(df_type,i)
    list_type_lp.append(dici)
    
# 数据类型转换
df_type_lp = pd.DataFrame(list_type_lp)

# 提取出前10名 烂片的类型
type_lp_top10 = df_type_lp.sort_values(by='type_lp_pre',ascending=False).iloc[:10]

# # 利用bokeh 制图
# type_lp_top10['size'] = type_lp_top10['typecount']**0.5*2  # 设置横轴的大小
# source = ColumnDataSource(type_lp_top10)  # 创建绘图数据

# list_type = type_lp_top10['typename'].tolist()
# hover = HoverTool(tooltips=[('数据量','@typecount'),('烂片率','@type_lp_pre')])

# # 设置标签内容
# output_file('lp_type.html') # 会给我们自动生成一个 html 文档
# p = figure(x_range = list_type,
#           plot_width=900,
#           plot_height=500,
#           title = '近期观影避雷针',
#           tools = [hover,'reset,xwheel_zoom,pan,crosshair,box_select'])
# p.circle(x='typename',y='type_lp_pre',source=source,size='size',
#         line_color='black',line_dash=[6,4],fill_color='red',fill_alpha=0.5)

# # 整了半天 终于开始作图了
# show(p)

print(df_type_lp)

    type_lp_pre  typecount typename
0      0.000000          1       武侠
1      0.298507         67       喜剧
2      0.000000          1       儿童
3      0.500000          2       古装
4      0.000000          9       家庭
5      0.100000         10       传记
6      0.250000         20       恐怖
7      0.250000         36       惊悚
8      0.218750         32       科幻
9      0.000000          5       灾难
10     0.100000         10       历史
11     0.227642        123       剧情
12     0.333333         48       爱情
13     0.000000          4       运动
14     0.312500         32       奇幻
15     0.000000          6       音乐
16     0.033333         30       动画
17     0.121212         33       冒险
18     0.222222          9       战争
19     1.000000          2       情色
20     0.281250         32       悬疑
21     0.324324         37       犯罪
22     0.306452         62       动作
23     0.000000          6       同性
24     0.000000          1       歌舞


In [93]:
'''
3.什么样的国家的电影更容易产生烂片？
'''

# 国家数据清洗
loc_list01 = []
for i in df['制片国家'].str.replace(' ','').str.split('/'):
    loc_list01.extend(i)

loc_list02 = []
for n in loc_list01:
    if n.rfind('话') or n.rfind('语'):
        rr = n.split(',')[0]
        loc_list02.append(rr)
    else:
        loc_list02.append(n)
    
loc_result = list(set(country_list02))

# 自定义函数
def lp_loc(data,typei):
    dic_loc_lp = {}
    datai = data[data['制片国家'].str.contains(typei)]
    lp_pre_i = len(datai[datai['豆瓣评分']<5.9]) / len(datai)
    
    # 总结输出 烂片信息
    dic_loc_lp['loc'] = typei
    dic_loc_lp['loccount'] = len(datai)
    dic_loc_lp['loc_lp_pre'] = lp_pre_i
    
    return dic_loc_lp

df_loc = df[['电影名称','制片国家','豆瓣评分']][df['制片国家'].notnull()]

list_loc_lp = []
for i in loc_result:
    dici = lp_loc(df_loc,i)
    list_loc_lp.append(dici)
    
df_loc_lp = pd.DataFrame(list_loc_lp)
df_loc_lp = df_loc_lp[df_loc_lp['loccount']>=3]
df_loc_lp[1:]


,主演,电影数量,烂片数量,烂片比例
0,中岛健人,1,1,1.0
1,冯绍峰,1,1,1.0
2,博朗·帕拉雷,1,1,1.0
3,古天乐,1,1,1.0
4,吴君如,1,1,1.0
5,吴镇宇,1,1,1.0
6,周渝民,1,1,1.0
7,坂泰斗,1,1,1.0
8,塔伦·埃格顿,2,1,0.5
9,大鹏,2,1,0.5


In [114]:
'''
4.卡司数量是否和烂片有关(要求)？
'''

df['演员人数'] = df['演员姓名'].str.replace(' ','').str.split(',').str.len()

df_leaderole01 = df[['演员人数','豆瓣评分']].groupby('演员人数').count()
df_leaderole02 = df[['演员人数','豆瓣评分']][df['豆瓣评分']<5.9].groupby('演员人数').count()

df_leaderole_pre = pd.merge(df_leaderole01,df_leaderole02,left_index=True,right_index=True)
df_leaderole_pre.columns = ['电影数量','烂片数量']

df_leaderole_pre.reset_index(inplace=True)
df_leaderole_pre['演员人数分组'] = pd.cut(df_leaderole_pre['演员人数'],
                                       [0,2,4,6,9,50],
                                       labels = ['1-2人','3-4人','5-6人','7-9人','10人以上'])

df_leaderole_pre2 = df_leaderole_pre[['演员人数分组','电影数量','烂片数量']].groupby('演员人数分组').sum()
df_leaderole_pre2['烂片比例'] = df_leaderole_pre2['烂片数量'] / df_leaderole_pre2['电影数量']

df_leaderole_pre2.reset_index(inplace=True)

df_leaderole_pre2

,演员人数分组,电影数量,烂片数量,烂片比例
0,1-2人,0,0,NaN
1,3-4人,11,3,0.272727
2,5-6人,29,7,0.241379
3,7-9人,39,8,0.205128
4,10人以上,165,44,0.266667


In [111]:
'''
5.分析主演 与 烂片之间的关系
'''
# 提取 主演
df['主演'] = df['演员姓名'].str.split(',').str[0]
df_pro01 = df[['主演','豆瓣评分']].groupby('主演').count()
df_pro02 = df[['主演','豆瓣评分']][df['豆瓣评分']<5.9].groupby('主演').count()

df_pro = pd.merge(df_pro01,df_pro02,left_index=True,right_index=True)
df_pro.columns = ['电影数量','烂片数量']

df_pro.reset_index(inplace=True)

df_pro['烂片比例'] = df_pro['烂片数量'] / df_pro['电影数量']

df_pro


,主演,电影数量,烂片数量,烂片比例
0,中岛健人,1,1,1.0
1,冯绍峰,1,1,1.0
2,博朗·帕拉雷,1,1,1.0
3,古天乐,1,1,1.0
4,吴君如,1,1,1.0
5,吴镇宇,1,1,1.0
6,周渝民,1,1,1.0
7,坂泰斗,1,1,1.0
8,塔伦·埃格顿,2,1,0.5
9,大鹏,2,1,0.5


In [112]:
'''
6. 分析导演与烂片之间的关系？
'''

#由于没有 计算过导演烂片比率 所以这里对 导演做个筛选
df_dir01 = df[['导演姓名','豆瓣评分']].groupby('导演姓名').count()
df_dir02 = df[['导演姓名','豆瓣评分']][df['豆瓣评分']<5.9].groupby('导演姓名').count()

df_dir = pd.merge(df_dir01,df_dir02,right_index=True,left_index=True)

df_dir.columns = ['电影数量','烂片数量']

df_dir.reset_index(inplace=True)

df_dir['烂片比率'] = df_dir['烂片数量'] / df_dir['电影数量']


,导演姓名,电影数量,烂片数量,烂片比率
0,Vaughn Stein,1,1,1.0
1,丁晟,1,1,1.0
2,三浦大辅,1,1,1.0
3,中村哲平,1,1,1.0
4,乔可·安华,1,1,1.0
5,任鹏远,1,1,1.0
6,伊恩·塞缪尔斯,1,1,1.0
7,佐藤信介,1,1,1.0
8,何念,1,1,1.0
9,克林特·伊斯特伍德,1,1,1.0


In [115]:
'''
6. 分析烂片电影的 必要因素，建立模型，判断是不是烂片？
    -- 必要因素
        * 电影类型 df_type_lp
        * 导演姓名 df_pro
        * 演员人数 df_leaderole_pre2
        * 主演姓名 df_dir
'''

data_lp = df[['电影类型','导演姓名','演员姓名','豆瓣评分']][df['豆瓣评分']<5.9]

data_lp.head()




,电影类型,导演姓名,演员姓名,豆瓣评分
8,"动作, 惊悚",帕诺斯·科斯马图斯,"尼古拉斯·凯奇, 安德丽亚·瑞斯波罗格, 莱纳斯·罗彻, 奈德·丹内利, 欧文·富尔, 理查...",5.5
25,"爱情, 情色",三浦大辅,"松坂桃李, 真飞圣, 富手麻妙, 猪塚健太, 樱井由纪, 小柳友, 马渕英俚可, 荻野友里,...",5.3
30,"喜剧, 动作, 犯罪",弗兰克·盖思堂彼得,"弗兰克·盖思堂彼得, 马利克·班泽拉, 伯纳尔·法西, 塞尔维托·埃斯波西多, 萨巴纳·奥扎...",5.5
35,动画,"李豪凌, 易小星, 竹内良贵","坂泰斗, 寿美菜子, 白石晴香, 安元洋贵, 大冢刚央, 长谷川育美, 白客, 彭师欣, 唐...",5.5
36,"喜剧, 爱情",田羽生,"韩庚, 郑恺, 于文文, 曾梦雪, 罗米",5.5
